In [2]:
#@title Import Statements
import pandas as pd
import sys

import tensorflow as tf
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pickle
from os import path
from keras.callbacks import ModelCheckpoint

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
RANDOM_SEED=2245
from numpy.random import seed
seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

import matplotlib.pyplot as plt

gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.keras.layers import (Input, GlobalAvgPool1D, Dense, Bidirectional, GRU, Dropout)

In [5]:
PATH_SCRIPT = "/content/drive/MyDrive/Audio classification/Notebooks-Mahim"
os.chdir(PATH_SCRIPT)
print(os.getcwd())

# X_train_file = "/content/drive/MyDrive/Audio classification/mfccs/train_13_2048_512.pkl"
# Y_train_file = "/content/drive/MyDrive/Audio classification/lables/train.pkl"

X_train_file = "/content/drive/MyDrive/Audio classification/mfccs/val_16_2048_512.pkl"
Y_train_file = "/content/drive/MyDrive/Audio classification/lables/val_16_2048_512.pkl"

with open(X_train_file, mode='rb') as fin:
    # X_train = pickle.load(fin)
    X = pickle.load(fin)
    X = np.asarray(X)

with open(Y_train_file, mode='rb') as fin:
    # Y_train = pickle.load(fin)
    Y = pickle.load(fin)
    Y = np.asarray(Y)
    Y = tf.keras.utils.to_categorical(Y)


print(X.shape, Y.shape)


################################################################################
## Load accordingly
################################################################################
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, 
                                    test_size=0.2, random_state=RANDOM_STATE)

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

/content/drive/.shortcut-targets-by-id/1DGiPjXTUI4ejRejy9C8oe3nAQXI6AZIW/Audio classification/Notebooks-Mahim
(8025, 1292, 16) (8025, 5)
(6420, 1292, 16) (6420, 5)
(1605, 1292, 16) (1605, 5)


In [ ]:
MODEL_SAVE_DIRECTORY = "Transformers"
DATE = "June29"
FILE_MODEL_PATH = "{}/{}-Transformer.h5".format(MODEL_SAVE_DIRECTORY, DATE)
FILE_MODEL_CHECKPOINT_BEST = "{}/{}-BestCKPT.h5".format(MODEL_SAVE_DIRECTORY, DATE)

print(f"FILE_MODEL_PATH = {FILE_MODEL_PATH}\nFILE_MODEL_CHECKPOINT_BEST = {FILE_MODEL_CHECKPOINT_BEST}")

In [6]:
#@title Helper Script is kept here verbatim. Please Update from GitHub regularly !!

from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import seaborn as sns

RANDOM_STATE = 2245

LABELS_MAPPING_FORWARD = {
    'national': 0,
    'international': 1,
    'economics': 2,
    'entertainment': 3,
    'sports': 4
}

LABELS_MAPPING_REVERSE = {
    0: 'national', 
    1: 'international', 
    2: 'economics', 
    3: 'entertainment', 
    4: 'sports'
}

NUM_LABELS = len(LABELS_MAPPING_FORWARD)

get_label_name = lambda label_id: LABELS_MAPPING_REVERSE.get(label_id)
get_label_class_id = lambda name: LABELS_MAPPING_REVERSE.get(name)

def get_various_metrics_and_print(Y_true, Y_predicted):
    TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
    accuracy = (TP + TN)/(TP+TN+FP+FN)
    recall = (TP)/(TP + FN)
    specificity = (TN)/(TN + FP) # TNR
    false_positive_rate = (FP)/(TN + FP) # false_positive_rate = 1 - TNR
    precision = (TP)/(TP + FP)
    false_discovery_rate = (FP)/(TP + FP)
    neg_predicted_val = (TN)/(TN + FN)
    f1_score = 2*((precision * recall) / (precision + recall))

    print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
    print("Accuracy = ", accuracy*100, "%")
    print("TPR = Sensitivity = Recall = ", recall*100, "%")
    print("TNR = Specificity = ", specificity*100, "%")
    print("Precision = PPV = Positive Predictive Value = ", precision*100, "%")
    print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
    print("FPR = False Positive Rate = ", false_positive_rate*100, "%")
    print("F1 Score = ", f1_score*100, "%")
    print("Neg Predicted Val = ", neg_predicted_val*100, "%")
    print("\n")
    # c_report = classification_report(y_true=Y_true, y_pred=Y_predicted)
    c_report = None
    print(c_report)
    return precision, recall, f1_score, c_report


def plot_confusion_matrix(Y_true=None, Y_predicted=None):
  ## https://www.youtube.com/watch?v=T27WVIM8Xys
    # mat = confusion_matrix(y_test,y_preds)
    cf_matrix = confusion_matrix(Y_true, Y_predicted)
    # cf_matrix = np.array([[23, 5], [3, 30]])
    print(cf_matrix)
    
    # group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_names = ['TN','FP','FN','TP']
    
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
    
    # labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]

    labels = np.asarray(labels).reshape(2,2)
    
    # sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
    #         fmt='.2%', cmap=plt.cm.magma)

    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap=plt.cm.magma)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")

In [91]:
## Checkpoints and other stuffs.
h5_name = FILE_MODEL_CHECKPOINT_BEST
# os.path.join("SAVED_THINGS", "transformer.h5")

checkpoint_single_path = ModelCheckpoint(
        h5_name,
        monitor="val_accuracy",
        verbose=1,
        save_best_only=True,
        mode="max",
        save_weights_only=True,
        # overwrite=True
)

### Transformer For Audio Classification codes taken from the following GitHub [repository](https://github.com/facundodeza/transfomer-audio-classification/blob/master/audio_classification_transformer.ipynb)

In [84]:
# @title Full Transformer Code from above GitHub repository

def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)


class MyMultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, **kwargs): # name='multi_head_attention'
    super(MyMultiHeadAttention, self).__init__(**kwargs)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def get_config(self):
    config = super().get_config().copy()
    config.update({
        'd_model': self.d_model,
        'num_heads': self.num_heads
    })
    return config

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model, **kwargs): # , name="PositionalEncoding"):
    super(PositionalEncoding, self).__init__(**kwargs)
    self.position = position
    self.d_model = d_model
    self.pos_encoding = self.positional_encoding(position, d_model)
    # self.name = name

  def get_config(self):
    config = super().get_config().copy()
    config.update({
        'position': self.position,
        'd_model': self.d_model
        # 'name': self.name
    })
    return config

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

    
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MyMultiHeadAttention(d_model, num_heads, name="attention")(
        {
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
        }
    )
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('projection layer is linear')
  
  else:
    projection=tf.identity(inputs) # Same layer repeat ?
    print('projection layer is none')
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model, name="PositionalEncoding")(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size, use_bias=True, activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

## https://stackoverflow.com/questions/58678836/notimplementederror-layers-with-arguments-in-init-must-override-get-conf


class CustomLearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000, name=None):
        super(CustomLearningRateScheduler, self).__init__()

        self.d_model = d_model

        ## https://github.com/tensorflow/tensorflow/issues/28799
        # Basically, can't keep in compile time.
        # self.d_model = tf.cast(self.d_model, tf.float32).numpy() ## Casting makes it unserializable ?
        # self.d_model = tf.cast(self.d_model, tf.float32)## Casting makes it unserializable ?

        self.warmup_steps = warmup_steps
        self.name = name  # Modified from the source

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        # return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
        return tf.math.rsqrt(tf.cast(self.d_model, tf.float32)) * tf.math.minimum(arg1, arg2) # recast in runtime

    def get_config(self):  # Modified from the source
        return {
            "d_model": self.d_model,
            "warmup_steps": self.warmup_steps,
            "name": self.name,
        }

## Hyperparameter Settings

In [120]:
EPOCHS = 50

NUM_LAYERS = 6
D_MODEL = X.shape[2] # eg. X.shape = (7711, 1292, 13) -> D_MODEL = 13
NUM_HEADS = 4
UNITS = 1024
DROPOUT = 0.1
TIME_STEPS = X.shape[1] # TIME_STEPS -> 1292
OUTPUT_SIZE = NUM_LABELS
# EXPERIMENTS=10
projection = 'none'

print(f"d_model = {D_MODEL}, num_heads = {NUM_HEADS}")

d_model = 16, num_heads = 2


In [122]:
model = transformer(
    time_steps=TIME_STEPS,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    output_size=OUTPUT_SIZE,  
    projection=projection
)

# CKPT_DIRECTORY = "CKPTS-June-27"
# checkpoint = ModelCheckpoint(CKPT_DIRECTORY, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') ## need to monitor val_get_f1 ?
callbacks_list = [checkpoint_single_path] # , callback_lr]

# CustomLearningRateScheduler(d_model=D_MODEL)

lr_schedule = CustomLearningRateScheduler(d_model=D_MODEL)
optimizer_adam_custom = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer_adam_custom,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# print(model.summary())
history = model.fit(X_train, Y_train, epochs=EPOCHS, validation_data=(X_val, Y_val), callbacks=callbacks_list, verbose=1)

model.save(FILE_MODEL_PATH)
print(f"Model Saved to {FILE_MODEL_PATH}")

projection layer is none
Saved
Epoch 1/2
1/1 [==============================] - 5s 5s/step - loss: 1.6021 - accuracy: 0.4000 - val_loss: 2.4311 - val_accuracy: 0.2000

Epoch 00001: val_accuracy did not improve from 0.30000
Epoch 2/2
1/1 [==============================] - 2s 2s/step - loss: 1.7706 - accuracy: 0.3000 - val_loss: 2.4128 - val_accuracy: 0.2000

Epoch 00002: val_accuracy did not improve from 0.30000


## Testing loading and reload weights

In [38]:
new_model = keras.models.load_model(FILE_MODEL_PATH, 
custom_objects={
    'PositionalEncoding': PositionalEncoding,
    'MyMultiHeadAttention': MyMultiHeadAttention
})

In [39]:
new_model.load_weights(h5_name)

## Testing

In [ ]:
# print(np.unique(np.argmax(Y, axis=-1), return_counts=True))

# Y_pred_test = model.predict(X_test)


# Y_test_labeled = np.argmax(Y_test, axis=-1)
# Y_test_labeled = [get_label_name(id) for id in Y_test_labeled]
# print(Y_test_labeled)

# Y_pred_labeled = np.argmax(Y_pred_test, axis=-1)
# Y_pred_labeled = [get_label_name(id) for id in Y_pred_labeled]
# print(Y_pred_labeled)

# c_report = classification_report(y_true=Y_test_labeled, y_pred=Y_pred_labeled)
# print(c_report)

In [ ]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()